In [ ]:
import requests
import time
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import smtplib
from email.mime.text import MIMEText

# OpenWeatherMap API Configuration
API_KEY = '2bff56c00d1e2e022b04a3f354e94c26'  # Your API key
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'
CITIES = ['Delhi', 'Mumbai', 'Chennai', 'Bangalore', 'Kolkata', 'Hyderabad']
UNITS = 'metric'  # Celsius

# Thresholds and settings
ALERT_THRESHOLD_TEMP = 35.0  # Alert if temp exceeds 35°C
CONSECUTIVE_ALERT_COUNT = 2  # Trigger alert after two consecutive updates
POLL_INTERVAL = 300  # Poll every 5 minutes (300 seconds)

# Store data for rollups
weather_data = []

# Function to get current weather data for a city
def get_weather(city):
    try:
        url = f"{BASE_URL}?q={city}&appid={API_KEY}&units={UNITS}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return {
                'city': city,
                'main': data['weather'][0]['main'],
                'temp': data['main']['temp'],
                'feels_like': data['main']['feels_like'],
                'humidity': data['main']['humidity'],
                'wind_speed': data['wind']['speed'],
                'dt': datetime.now()  # Use system time to capture real-time updates
            }
        else:
            print(f"Error retrieving data for {city}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception occurred for {city}: {e}")
        return None

# Function to fetch weather data continuously at intervals
def fetch_weather_data(interval=POLL_INTERVAL):
    global weather_data
    while True:
        for city in CITIES:
            data = get_weather(city)
            if data:
                weather_data.append(data)
                print(f"Weather data for {city}: {data}")
        # Process data every interval
        if weather_data:  # Only process if there is data
            process_weather_data(weather_data)
            check_alerts(weather_data)
        time.sleep(interval)

# Function to calculate daily weather summary (rollups)
def calculate_daily_summary(weather_data):
    df = pd.DataFrame(weather_data)
    
    # Check if 'dt' is in the data, handle missing data safely
    if 'dt' not in df.columns:
        print("No valid weather data to process.")
        return None

    df['date'] = df['dt'].dt.date  # Extract date
    
    # Daily rollups and aggregation
    daily_summary = df.groupby(['city', 'date']).agg({
        'temp': ['mean', 'max', 'min'],
        'humidity': 'mean',
        'wind_speed': 'mean',
        'main': lambda x: x.mode()[0]  # Dominant weather condition
    }).reset_index()

    # Rename columns
    daily_summary.columns = ['city', 'date', 'avg_temp', 'max_temp', 'min_temp', 'avg_humidity', 'avg_wind_speed', 'dominant_weather']
    
    # Save to CSV (optional)
    daily_summary.to_csv('daily_weather_summary.csv', index=False)
    
    return daily_summary

# Function to process weather data and calculate rollups
def process_weather_data(weather_data):
    daily_summary = calculate_daily_summary(weather_data)
    if daily_summary is not None:
        print(daily_summary)
    weather_data.clear()  # Reset data for next round

# Function to send email alert
def send_alert(city, temp):
    msg = MIMEText(f"Alert! {city} temperature exceeded {ALERT_THRESHOLD_TEMP}°C. Current temp: {temp}°C")
    msg['Subject'] = f"Weather Alert for {city}"
    msg['From'] = "pavithrapalni1282003@gmail.com"  # Your email
    msg['To'] = "pavithrapriya282003@gmail.com"  # Recipient email (Make sure to include the domain)

    # Replace this with your email provider's SMTP server and port
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()  # Start TLS encryption
        server.login("pavithrapalni1282003@gmail.com", "Pavithra@2003")  # Replace with your Gmail and password
        server.send_message(msg)

    print(f"Email alert sent for {city}")


# Function to check for alerts
def check_alerts(weather_data):
    df = pd.DataFrame(weather_data)
    if 'temp' not in df.columns or df.empty:
        print("No temperature data available for alert checking.")
        return

    df = df.sort_values(by=['city', 'dt'])
    
    for city in CITIES:
        city_data = df[df['city'] == city]
        breaches = city_data[city_data['temp'] > ALERT_THRESHOLD_TEMP]
        
        if len(breaches) >= CONSECUTIVE_ALERT_COUNT:
            send_alert(city, breaches.iloc[-1]['temp'])

# Start fetching real-time data
fetch_weather_data()


Error retrieving data for Delhi: 401
Error retrieving data for Mumbai: 401
Error retrieving data for Chennai: 401
Error retrieving data for Bangalore: 401
Error retrieving data for Kolkata: 401
Error retrieving data for Hyderabad: 401
Weather data for Delhi: {'city': 'Delhi', 'main': 'Haze', 'temp': 35.05, 'feels_like': 33.65, 'humidity': 24, 'wind_speed': 2.57, 'dt': datetime.datetime(2024, 10, 18, 14, 57, 35, 429399)}
Weather data for Mumbai: {'city': 'Mumbai', 'main': 'Haze', 'temp': 30.99, 'feels_like': 37.57, 'humidity': 70, 'wind_speed': 4.63, 'dt': datetime.datetime(2024, 10, 18, 14, 57, 35, 538759)}
Weather data for Chennai: {'city': 'Chennai', 'main': 'Haze', 'temp': 31.31, 'feels_like': 38.31, 'humidity': 80, 'wind_speed': 2.06, 'dt': datetime.datetime(2024, 10, 18, 14, 57, 35, 648202)}
Weather data for Bangalore: {'city': 'Bangalore', 'main': 'Clouds', 'temp': 26.72, 'feels_like': 27.99, 'humidity': 64, 'wind_speed': 1.54, 'dt': datetime.datetime(2024, 10, 18, 14, 57, 35, 75